# MILP for COPP

In [259]:
# import modules
from gurobipy import *
import numpy as np
import pandas as pd
import pyodbc
import re

In [260]:
# connect to the database
conn_str = 'DSN=JDATA;database=WarehouseModel;Trusted_Connection=yes;'
conn = pyodbc.connect(conn_str)
cursor = conn.cursor()

In [261]:
# get data from database
lineItem = pd.read_sql("SELECT ItemID FROM LineItem", conn)
pickers = pd.read_sql("SELECT PickerID, Speed, Capacity, PlaceTime FROM Pickers", conn)
transporters = pd.read_sql("SELECT TransporterID, Speed, Capacity, UnloadTime FROM Transporters", conn)
Sku_L = pd.read_sql("SELECT LineItem.ItemID, Skus.PickNodeID FROM LineItem INNER JOIN Skus ON LineItem.SkuID = Skus.SkuID", conn)
Nodes = pd.read_sql("SELECT ID, Node FROM Nodes WHERE NOT Type='MyTransferNode' AND NOT Type='MyDepotNode'", conn)
# dataframe and list of all physic nodes (Home, Picking locations, Handoff spots, Depot)
PhyNodes = pd.read_sql("SELECT Node FROM Nodes WHERE NOT Type='MyTransferNode'", conn)
PhyNodes_list = PhyNodes['Node'].tolist()
# get some setting paras
# number of items
NumItems = len(lineItem)
# number of depot replicas
DReps = 10
# depots dataframe
Depots = pd.DataFrame(list(range(1,DReps+1)), columns = ['Reps'])
# get travel time matrix from TimeMatrix.xlsx
Time_matrix = pd.read_excel('TimeMatrix.xlsx', sheet_name = 'TimeMatrix', header=None).fillna(0)

## Gurobi Model

In [262]:
# create empty model object
model = Model('COPP')
# set time limit for optimization
model.setParam('TimeLimit', 2000)

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf


### Parameters and keys

In [263]:
# define the speed, capacity, place time for each picker
_, Vp, Cp, PLT = multidict(pickers.set_index(['PickerID'])[['Speed','Capacity','PlaceTime']].T.to_dict('list'))
# define the speed, capacity, unload time for each transporter
_, Vt, Ct, ULT = multidict(transporters.set_index(['TransporterID'])[['Speed','Capacity','UnloadTime']].T.to_dict('list'))

# Some useful dataframes
P = pickers[['PickerID']]  # pickers
T = transporters[['TransporterID']] # transporters
I = lineItem  # items
HR = Nodes.drop(0) # all physical handoff spots
# all picking locations
RI = pd.DataFrame(Sku_L.apply(lambda x: 'MyPickNode'+str(x['PickNodeID'])+'_PL'+str(x['ItemID']), axis = 1), columns=['Node'])
HRI = pd.DataFrame() # all handoff replicas
#complete HRI df
for i in range(NumItems):
    HRi = HR.copy()
    HRi['Node'] = HRi['Node'] + '_I'+str(i+1)
    HRI = HRI.append(HRi,ignore_index=True)
# depot replicas
D = pd.DataFrame(Depots.apply(lambda x: 'MyDepot_'+ str(x['Reps']), axis =1), columns = ['Reps'])

## Decision Variables

### Makespan decision variable -- m

In [264]:
m =  model.addVar(vtype=GRB.CONTINUOUS, name = 'm')

### Assignment decision variables -- a[e1, e2, i]

In [265]:
# define the set of all items, pickers and transporters. For decision variable a[e1,e2,i]
# e1: picker
# e2: transporter
# i: item
I_P = pd.merge(I, P, how='cross')
I_P_T = pd.merge(I_P , T, how='cross')
I_P_T['Cell'] = 1
# create dicts for decision variable
Da, _ = multidict(I_P_T.set_index(['PickerID', 'TransporterID', 'ItemID']).to_dict()['Cell'])
# create the decision variable
a = model.addVars(Da, vtype=GRB.BINARY, name = 'A')

### Picker item sequence decision variables -- X[e1, i, j]

In [266]:
# define the set of all pickers and item picking sequence segments. For decision variabel X[e1,i,j]
# e1: picker
# i: picking item (item 0 is the virtual start item)
# l: next picking item (item -1 is the virtual end item)
I_I = pd.merge(I, I, how='cross')
I_I['Keep']=I_I.apply(lambda x: False if x['ItemID_x'] == x['ItemID_y'] else True, axis = 1)
I_I = I_I[I_I['Keep']].reset_index(drop=True)
# add virtual items sequences
for i in range(1,NumItems+1):
    I_I.loc[len(I_I.index)] = [0, i, True]
for i in range(1,NumItems+1):
    I_I.loc[len(I_I.index)] = [i,-1, True]    
P_I_I = pd.merge(P,I_I, how='cross')
P_I_I.columns = ['e1','i','j','Keep']

# create dicts for decision variable
DX, _ = multidict(P_I_I.set_index(['e1', 'i', 'j']).to_dict()['Keep'])
# create the decision variable
X = model.addVars(DX, vtype=GRB.BINARY, name = 'X')

### Transporter item sequence decision variables -- Z[e2,i,j]

In [267]:
# define the set of all pickers and item picking sequence segments. For decision variabel Z[e2,i,j]
# e2: transporter
# i: picking item (item 0 is the virtual start item)
# l: next picking item (item -1 is the virtual end item)
T_I_I = pd.merge(T, I_I, how='cross')
T_I_I.columns = ['e1','i','j','Keep']

# create dicts for decision variable
DZ, _ = multidict(T_I_I.set_index(['e1', 'i', 'j']).to_dict()['Keep'])
# create the decision variable
Z = model.addVars(DZ, vtype=GRB.BINARY, name = 'Z')


### Picker node sequence decision variables -- Y[e1, k ,k1]

In [268]:
# define the sets of all pickers and travel segements. For decision Variable Y[e1,k,k1]
# e1: picker
# k: previous node
# k1: next node (Picking location, Handoff spots replicas, Home node replicas)

# set when k1 is handoff replicas
# complete P_RI_HRI df
# HR: the set of physical handoff spots
RI_HRI = pd.merge(Sku_L,HR, how='cross')
RI_HRI['k'] = RI_HRI.apply(lambda x: 'MyPickNode'+str(x['PickNodeID'])+'_PL'+str(x['ItemID']), axis = 1)
RI_HRI['k1'] = RI_HRI.apply(lambda x: x['Node']+'_I'+str(x['ItemID']), axis = 1)
P_RI_HRI = pd.merge(P,RI_HRI, how='cross')[['PickerID','k','k1']]
P_RI_HRI.columns = ['e1','k','k1']

# set when k1 is Picking node
# HRI: the set of all handoff spots replicas
# RI: the set of all picking locations
HRI_RI = pd.merge(Sku_L,HRI, how='cross')
HRI_RI['k1'] = HRI_RI.apply(lambda x: 'MyPickNode'+str(x['PickNodeID'])+'_PL'+str(x['ItemID']), axis = 1)
droplist = []
for i in range(len(HRI_RI)):
    if re.findall("\d+",HRI_RI.at[i,'Node'])[-1] == str(HRI_RI.at[i,'ItemID']):
        droplist.append(i)
HRI_RI = HRI_RI.drop(droplist)
P_HRI_RI = pd.merge(P,HRI_RI, how='cross')[['PickerID','Node','k1']]
P_HRI_RI.columns = ['e1','k','k1']
Hp = P.copy()
Hp.columns = ['e1']
Hp['k'] = Hp.apply(lambda x: 'MyHome_P'+str(x['e1']), axis =1)
P_H_RI = pd.merge(Hp,RI,how='cross')
P_H_RI.columns = ['e1','k','k1']
P_HHRI_RI = pd.concat([P_HRI_RI, P_H_RI], ignore_index=True)

# set when k1 is home node replicas
P_HRI_H = pd.merge(Hp, RI_HRI['k1'].to_frame('k'), how='cross')[['e1','k_y','k_x']]
P_HRI_H.columns = ['e1','k','k1']

# get the final sets and dict
Y_P = pd.concat([P_RI_HRI, P_HHRI_RI, P_HRI_H], ignore_index=True)
Y_P['Cell'] = 1
DYp, _ = multidict(Y_P.set_index(['e1', 'k', 'k1']).to_dict()['Cell'])

# create the decision variable
Yp = model.addVars(DYp, vtype=GRB.BINARY, name = 'Yp')

### Transporter node sequence decision variables -- Y[e2, k ,k1]

In [269]:
# define a function that is used to decide wheteher a node seuqnece (HRI_HRI) is feasible
def T_NodeSeqFeasibility(row):
    head_item = row['Node_x'][row['Node_x'].index('_'):]
    tail_item = row['Node_y'][row['Node_y'].index('_'):]
    if head_item == tail_item:
        return (False)
    else:
        return(True)

In [270]:
# define the sets of all transporters and travel segements. For decision Variable Y[e2,k,k1]
# e2: transporter
# k: previous node
# k1: next node (Depot replicas, Handoff spots replicas, Home node replicas)

# set when k1 is depot replicas
HRI_D = pd.merge(HRI, D, how='cross')
T_HRI_D = pd.merge(T, HRI_D[['Node','Reps']], how='cross')
T_HRI_D.columns = ['e2','k','k1']

# # set when k1 is home node replicas
Ht = T.copy()
Ht.columns = ['e2']
Ht['k1'] = Ht.apply(lambda x: 'MyHome_T'+str(x['e2']), axis =1)
D_H = pd.merge(D, Ht, how='cross')
T_D_H = D_H[['e2','Reps','k1']]
T_D_H.columns = ['e2','k','k1']

# set when k1 is handoff spots replicas
# when k is home node
H_HRI = pd.merge(Ht, HRI, how='cross')
T_H_HRI = H_HRI[['e2','k1','Node']]
T_H_HRI.columns = ['e2','k','k1']
# when k is depot replicas
D_HRI = pd.merge(D, HRI, how='cross')
# D_HRI['k'] = D_HRI.apply(lambda x: 'DepotRep_'+ str(x['Reps']), axis =1)
T_D_HRI = pd.merge(T, D_HRI, how='cross')[['TransporterID','Reps','Node']]
T_D_HRI.columns = ['e2','k','k1']
# when k is another handoff spots
HRI_HRI = pd.merge(HRI, HRI, how='cross')
HRI_HRI['Keep']=HRI_HRI.apply(lambda x: T_NodeSeqFeasibility(x), axis = 1)
HRI_HRI = HRI_HRI[HRI_HRI['Keep']].reset_index(drop=True)[['Node_x','Node_y']]
T_HRI_HRI = pd.merge(T,HRI_HRI, how='cross')
T_HRI_HRI.columns = ['e2','k','k1']

# get the final sets and dict
Y_T = pd.concat([T_H_HRI, T_HRI_D, T_D_HRI, T_D_H, T_HRI_HRI], ignore_index=True)
Y_T['Cell'] = 1
DYt, _ = multidict(Y_T.set_index(['e2', 'k', 'k1']).to_dict()['Cell'])

# create the decision variable
Yt = model.addVars(DYt, vtype=GRB.BINARY, name = 'Yt')

In [271]:
# define the function that set the time value for each node sequence
def setTimeforTLs(row):
    head_node = row['k'][:row['k'].index('_')]
    tail_node = row['k1'][:row['k1'].index('_')]
    head_id = PhyNodes_list.index(head_node)
    tail_id = PhyNodes_list.index(tail_node)
    return(Time_matrix.iloc[head_id, tail_id])


Y_P['Time'] = Y_P.apply(lambda x: setTimeforTLs(x), axis = 1)
# define travel time for each possible node sequence for picker
_, Lp = multidict(Y_P.set_index(['k', 'k1']).to_dict()['Time'])

Y_T['Time'] = Y_T.apply(lambda x: setTimeforTLs(x), axis = 1)
_, Lt = multidict(Y_T.set_index(['k', 'k1']).to_dict()['Time'])

### Transporter payload at node decision variables -- S[e2, k, k1]

In [272]:
# create the decision variable
St = model.addVars(DYt, vtype=GRB.INTEGER, name = 'St')

### Picker arrival time decision variables -- t[e1, k]

In [273]:
# All nodes that a picker may visit (exclude home node)
P_AllNodes = pd.concat([RI, HRI[['Node']]], ignore_index=True)
tp_df = pd.merge(pickers[['PickerID','Capacity']],P_AllNodes, how='cross' )
# create dicts for decision variable
Dtp, _ = multidict(tp_df.set_index(['PickerID','Node']).to_dict()['Capacity'])
# create the decision variable
tp = model.addVars(Dtp, vtype=GRB.CONTINUOUS, lb=0, name = 'tp')

### Transporter arrival time decision variables -- t[e2, k]

In [274]:
# All nodes that a transporter may visit (exclude home node)
NewD = D.copy()
NewD.columns = ['Node']
T_AllNodes = pd.concat([NewD, HRI[['Node']]], ignore_index=True)
tt_df = pd.merge(transporters[['TransporterID','Capacity']],T_AllNodes, how='cross')
# create dicts for decision variable
Dtt, _ = multidict(tt_df.set_index(['TransporterID','Node']).to_dict()['Capacity'])
# create the decision variable
tt = model.addVars(Dtt, vtype=GRB.CONTINUOUS, lb=0, name = 'tt')

### Some useful lists and dicts for building constraints

In [275]:
# Picking Locations list
PLsL = RI['Node'].tolist()
# all handoff replicas list
HRsL = HRI['Node'].tolist()
# pickers list
PsL = pickers['PickerID'].tolist()
# pikcer home list
PHL = Hp['k'].tolist()
# transporter list
TsL = transporters['TransporterID'].tolist()
# transporter home list
THL = Ht['k1'].tolist()
# all items list (include virtual items)
IsL = lineItem['ItemID'].tolist()
IsL.extend([0,-1])
# all depot replicas list
DpsL = D['Reps'].tolist()
# bigM for all time related constraints
BigMtime = 10000

## Constraints

In [276]:
# constraint No.1
for e2 in TsL:
    for k in DpsL:
        model.addConstr(m >=tt[e2,k])

In [277]:
# constraint No.2a
# each picking location will only be visited by one picker
# each item
for i in range(1,NumItems+1):
    Ri = PLsL[i-1]
    expr = LinExpr()
    # each picker
    for e1 in PsL:
        klist = Y_P[(Y_P['k1'] == Ri) & (Y_P['e1'] == e1)]['k'].tolist()
        expr.add(quicksum(Yp[e1,k,Ri] for k in klist))
    # add constraints
    model.addConstr(expr ==1)

In [278]:
# constraint No.2b
# after visiting a picking location, the picker will move to one of the handoff replicas of that item
# each item
for i in range(1,NumItems+1):
    Ri = PLsL[i-1]
    # each picker
    for e1 in PsL:
        # the possible next node list
        HRi = RI_HRI['k1'][RI_HRI['ItemID'] == i].tolist()
    # add constraints
    model.addConstr(quicksum(quicksum(Yp[e1,Ri,k1] for k1 in HRi) for e1 in PsL) ==1)

In [279]:
# constraint No.2c 2d
# a picker will be back to its home node after picking all needed items
# each picker
for e1 in PsL:
    # home node for the picker
    HNe1 =  PHL[e1-1]
    # 2c a picker will be back to its home node after picking all needed items
    model.addConstr(quicksum(Yp[e1,k,HNe1] for k in HRsL) == 1)
    # 2d a picker will start from its home node
    model.addConstr(quicksum(Yp[e1,HNe1,k1] for k1 in PLsL) == 1)

In [280]:
# constraints No.3
# each handoff node
k1list = HRsL + DpsL
for k1 in k1list:
    e2klist = Y_T[Y_T['k1'] == k1][['e2','k']].values.tolist()
    # constraint 3.a
    # each handoff node and depot replica will be visited by no more than one transporter
    model.addConstr(quicksum(Yt[e2,k,k1] for e2,k in e2klist) <= 1)

In [281]:
# constraints No.3
# each transporter
for e2 in TsL:
    # each handoff node
    k1list = HRsL + DpsL
    for k1 in k1list:
        klist = Y_T[(Y_T['k1'] == k1) & (Y_T['e2'] == e2)]['k'].tolist()
        k2list = Y_T[(Y_T['k'] == k1) & (Y_T['e2'] == e2)]['k1'].tolist()
        # 3.b when a transporter enter a handoff node or a depot replica, constarint ensures that it will leave that node
        model.addConstr(quicksum(Yt[e2,k,k1] for k in klist) == quicksum(Yt[e2,k1,k2] for k2 in k2list))

In [282]:
# contraint No.3
for e2 in TsL:
    HNe2 = THL[e2-1]
    # constraints 3.c  a transporter will be back to its home node
    model.addConstr(quicksum(Yt[e2,k,HNe2] for k in DpsL) == 1)
    # constraint 3.d  a transporter will start from its home node
    model.addConstr(quicksum(Yt[e2,HNe2,k1] for k1 in HRsL) == 1)

In [283]:
# constraints No.3e
# for an item, it can only be placed at one handoff node by one transporter
# each transporter
# each item
for i in range(1,NumItems+1):
    HRi = RI_HRI['k1'][RI_HRI['ItemID'] == i].tolist()
    expr = LinExpr()
    for e2 in TsL:
        for k1 in HRi:
            klist = Y_T[(Y_T['k1'] == k1) & (Y_T['e2'] == e2)]['k'].tolist()
            expr.add(quicksum(Yt[e2,k,k1] for k in klist))
    model.addConstr(expr == 1)

In [284]:
# constraint No.3f
# to set appropriate node sequence of picker based on X[e1,i,j]
BigM = 2
# each item i
for i in range(1,NumItems+1):
    jlist = list(range(1,NumItems+1))
    jlist.remove(i)
    kk1list = RI_HRI['k1'][RI_HRI['ItemID'] == i].tolist()
    # each picker       
    for e1 in PsL:
        HNe1 = PHL[e1-1]
        # add constraints 19.b
        # each node of HRi
        for kk1 in kk1list:
            model.addConstr(Yp[e1,PLsL[i-1],kk1] <= Yp[e1,kk1,HNe1] + BigM*(1-X[e1,i,-1]))
            model.addConstr(Yp[e1,kk1,HNe1] <= Yp[e1,PLsL[i-1],kk1] + BigM*(1-X[e1,i,-1]))
        # the next item
        for j in jlist:
            # each node of HRi
            for kk1 in kk1list:
            # add constraints 19.a
                model.addConstr(Yp[e1,PLsL[i-1],kk1] <= Yp[e1,kk1,PLsL[j-1]] + BigM*(1-X[e1,i,j]))
                model.addConstr(Yp[e1,kk1,PLsL[j-1]] <= Yp[e1,PLsL[i-1],kk1] + BigM*(1-X[e1,i,j]))  

In [285]:
# constaint No.4
# each item can only be retrieved by one picker and be placed on one transporter
for i in range(1,NumItems+1):
    model.addConstr(quicksum(quicksum(a[e1,e2,i] for e2 in TsL) for e1 in PsL) == 1)

In [286]:
# constraint No.5 and No.6
# the constarints used to set appropriate value of a[e1,e2,i] to pair picker and transporter
# each item 
BigM = 2
for i in range(1,NumItems+1):
    Ri = PLsL[i-1]
    HRi = RI_HRI['k1'][RI_HRI['ItemID'] == i].tolist()
    for e1 in PsL:
        for e2 in TsL:
            expr = LinExpr()
            for k1 in HRi:
                klist = Y_T[(Y_T['k1'] == k1) & (Y_T['e2'] == e2)]['k'].tolist()
                expr.add(quicksum(Yt[e2,k,k1] for k in klist) + Yp[e1,Ri,k1])
                # No.6
                model.addConstr(Yp[e1,Ri,k1] <= quicksum(Yt[e2,k,k1] for k in klist) + BigM*(1-a[e1,e2,i]))
                model.addConstr(quicksum(Yt[e2,k,k1] for k in klist) <= Yp[e1,Ri,k1] + BigM*(1-a[e1,e2,i]))
            #No.5
            model.addConstr(2*a[e1,e2,i] <= expr)
            model.addConstr(a[e1,e2,i] + 1 >= expr)    

In [287]:
# constraint No.7a
# each item will only be picked by one picker
# each item
for i in range(1,NumItems+1):
    # the next item list
    jlist = IsL.copy()
    jlist.remove(i)
    jlist.remove(0)
    # add constraints
    model.addConstr(quicksum(quicksum(X[e1,i,j] for j in jlist) for e1 in PsL) == 1)

# each item
for j in range(1,NumItems+1):
    # the previous item list
    ilist = IsL.copy()
    ilist.remove(j)
    ilist.remove(-1)        
    # add constraints
    model.addConstr(quicksum(quicksum(X[e1,i,j] for i in ilist) for e1 in PsL) == 1) 

In [288]:
# constraint No.7b (18)
# each picker will start from picking item 0 and end from picking item -1
for e1 in PsL:
    model.addConstr(quicksum(X[e1,i,-1] for i in range(1,NumItems+1)) == 1)
    model.addConstr(quicksum(X[e1,0,j] for j in range(1,NumItems+1)) == 1)

In [289]:
# constraint No.7c (20)
# set the flow of the item sequence for picker
#each item
for j in range(1,NumItems+1):
    ilist = list(range(0,NumItems+1))
    ilist.remove(j)
    llist = list(range(1,NumItems+1))
    llist.remove(j)
    llist.append(-1)
    # each picker
    for e1 in PsL:
        model.addConstr(quicksum(X[e1,i,j] for i in ilist) == quicksum(X[e1,j,l] for l in llist))

In [290]:
# constraint No.8
# to set appropriate node sequence of picker based on X[e1,i,j]
# each item i
for i in range(1,NumItems+1):
    # the next item
    jlist = list(range(1,NumItems+1))
    jlist.remove(i)
    kk1list = RI_HRI['k1'][RI_HRI['ItemID'] == i].tolist()
    # each picker
    for e1 in PsL:
        HNe1 = PHL[e1-1]
        # the next item
        for j in jlist:      
            # add constraints
            model.addConstr(2*X[e1,i,j] <= quicksum(Yp[e1,PLsL[i-1],kk1]+Yp[e1,kk1,PLsL[j-1]] for kk1 in kk1list))
            model.addConstr(X[e1,i,j]+1 >= quicksum(Yp[e1,PLsL[i-1],kk1]+Yp[e1,kk1,PLsL[j-1]] for kk1 in kk1list))
        model.addConstr(2*X[e1,i,-1] <= quicksum(Yp[e1,PLsL[i-1],kk1]+Yp[e1,kk1,HNe1] for kk1 in kk1list))
        model.addConstr(X[e1,i,-1]+1 >= quicksum(Yp[e1,PLsL[i-1],kk1]+Yp[e1,kk1,HNe1] for kk1 in kk1list))
        model.addConstr(2*X[e1,0,i] <= quicksum(Yp[e1,PLsL[i-1],kk1] for kk1 in kk1list) + Yp[e1,HNe1,PLsL[i-1]])
        model.addConstr(X[e1,0,i]+1 >= quicksum(Yp[e1,PLsL[i-1],kk1] for kk1 in kk1list) + Yp[e1,HNe1,PLsL[i-1]])

In [291]:
# constraint No.9
# the time picker travel from picking node to handoff node
PKT = 30
# each picker
for e1 in PsL:
    # each item
    for i in range(1,NumItems+1):
        # the handoff replicas of item i
        HRi = RI_HRI['k1'][RI_HRI['ItemID'] == i].tolist()
        Ri = PLsL[i-1]
        # each handoff node
        for k in HRi:
            model.addConstr(tp[e1,k] >= tp[e1,Ri] + PKT + Lp[Ri,k]/Vp[e1] - BigMtime*(1-Yp[e1,Ri,k]))

In [292]:
# constraint No.10
# the time picker travel from handoff node to picking node
# each picker
for e1 in PsL:
    # each item
    for i in range(1,NumItems+1): 
        jlist = list(range(1,NumItems+1))
        jlist.remove(i)
        # the handoff replicas of item i
        HRi = RI_HRI['k1'][RI_HRI['ItemID'] == i].tolist()
        # the next item
        for j in jlist:
            Rj = PLsL[j-1]
            # each handoff node of item i
            for k in HRi:
                model.addConstr(tp[e1,Rj] >=tp[e1,k] + Lp[k,Rj]/Vp[e1] + PLT[e1] - BigMtime*(2-X[e1,i,j]-Yp[e1,k,Rj]))

In [293]:
# constraint No.11
# The time picker travel from home node to picking node
# each picker
for e1 in PsL:
    HNe1 = PHL[e1-1]
    # each item
    for i in range(1,NumItems+1):
        Ri = PLsL[i-1]
        model.addConstr(tp[e1,Ri] >= Lp[HNe1,Ri]/Vp[e1] - BigMtime*(1-Yp[e1,HNe1,Ri]))

In [294]:
# constraint No.12
# for each picker, its payload cannot exceed its capacity
k1list = HRsL + DpsL
for e2 in TsL:
    for k1 in k1list:
        klist =  Y_T[(Y_T['k1'] == k1) & (Y_T['e2'] == e2)]['k'].tolist()
        for k in klist:
            model.addConstr(St[e2,k,k1] <= Ct[e2]*Yt[e2,k,k1])

In [295]:
# constraint No.13
# when the transporter leave a node and is travelled from home node, its payload should be 1
for e2 in TsL:
    HNe2 = THL[e2-1]
    for k in HRsL:
        model.addConstr(St[e2,HNe2,k] == Yt[e2,HNe2,k])

In [296]:
# constraint No.14
# determines the payload carried by a transporter when picking up the first item after leaving a depot replica
for e2 in TsL:
    for k in DpsL:
        for k1 in HRsL:
            model.addConstr(St[e2,k,k1] == Yt[e2,k,k1])

In [297]:
# constraint No.15
# forces the payload weight to be at least as large as the summation of payload when e2 leaves node k1
# plus the item loaded at k2.

# physical handoff spot list (used for reduce computation time)
HRL =  ['']+HR['Node'].tolist()

for e2 in TsL:
    for k2 in HRsL:
        k2_itemid = int(k2[k2.index('_')+2:])
        # get the k1list
        k1list = HRsL.copy()
        HRi_k2 = []
        for i in range(1,len(HR)+1):
            ele = HRL[i] + '_I'+ str(k2_itemid)
            HRi_k2.append(ele)
            k1list.remove(ele)    
        for k1 in k1list:
            itemlist = list(range(1,NumItems+1))
            k1_itemid = int(k1[k1.index('_')+2:])
            itemlist.remove(k2_itemid)
            itemlist.remove(k1_itemid)
            # get klist
            klist = [THL[e2-1]] + DpsL
            for item in itemlist:
                for i in range(1,len(HR)+1):
                    ele = HRL[i] + '_I'+ str(item)
                    klist.append(ele) 
            model.addConstr(St[e2,k1,k2] >= quicksum(St[e2,k,k1] for k in klist) + 1 - Ct[e2]*(1-Yt[e2,k1,k2]))

In [298]:
# constraint No.16
# the time when a transporter travel from a handoff node
for i in range(1,NumItems+1):
    HRi = RI_HRI['k1'][RI_HRI['ItemID'] == i].tolist()
    for e2 in TsL:
        for k in HRi:
            k1list =  Y_T[(Y_T['k'] == k) & (Y_T['e2'] == e2)]['k1'].tolist()
            for k1 in k1list:
                for e1 in PsL:
                    model.addConstr(tt[e2,k1] >= tt[e2,k] + PLT[e1] + Lt[k,k1]/Vt[e2] - BigMtime*(2-Yt[e2,k,k1]-a[e1,e2,i]))

In [299]:
# constraints No.17
# the time when a transporter travel from a depot node
for e2 in TsL:
    for k in DpsL:
        for k1 in HRsL:
            model.addConstr(tt[e2,k1] >= tt[e2,k] + ULT[e2] + Lt[k,k1]/Vt[e2] - BigMtime*(1-Yt[e2,k,k1]))

In [300]:
# constraints No.18
# the time when a transporter travel from home node
for e2 in TsL:
    HNe2 = THL[e2-1]
    for k in HRsL:
        model.addConstr(tt[e2,k] >= Lt[HNe2,k]/Vt[e2] - BigMtime*(1-Yt[e2,HNe2,k]))        

In [301]:
# constraints No.19
# the transporter can only do the handoff process when picker is ready
PKT = 30

for i in range(1,NumItems+1):
    HRi = RI_HRI['k1'][RI_HRI['ItemID'] == i].tolist()
    for e1 in PsL:
        for k1 in HRi:
            klist = Y_P[(Y_P['k1'] == k1) & (Y_P['e1'] == e1)]['k'].tolist()
            for k in klist:
                for e2 in TsL:
                    model.addConstr(tt[e2,k1] >= tp[e1,k1] - BigMtime*(2-Yp[e1,k,k1]-a[e1,e2,i]))                

In [302]:
# constraints No.20
# picker cannot move to the next location from the assigned handoff node until the placement of the previous 
# item is completed
for i in range(1,NumItems+1):
    jlist = list(range(1,NumItems+1))
    jlist.remove(i)
    HRi = RI_HRI['k1'][RI_HRI['ItemID'] == i].tolist()
    for j in jlist:
        Rj = PLsL[j-1]
        for k in HRi:
            for e1 in PsL:
                for e2 in TsL:
                    model.addConstr(tp[e1,Rj] >= tt[e2,k] + PLT[e1] + Lp[k,Rj]/Vp[e1] - BigMtime*(3-X[e1,i,j]-a[e1,e2,i]-Yp[e1,k,Rj]))

In [303]:
obj = m
model.setObjective(obj, GRB.MINIMIZE)
# solve model
model.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 359054 rows, 181551 columns and 11362930 nonzeros
Model fingerprint: 0x8691571b
Variable types: 941 continuous, 180610 integer (91480 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+04]
Presolve removed 17280 rows and 11460 columns (presolve time = 7s) ...
Presolve removed 17280 rows and 11460 columns (presolve time = 10s) ...
Presolve removed 17460 rows and 11460 columns (presolve time = 15s) ...
Presolve removed 17460 rows and 11460 columns (presolve time = 22s) ...
Presolve removed 17460 rows and 11460 columns (presolve time = 27s) ...
Presolve removed 17460 rows and 11460 columns
Presolve time: 29.48s
Presolved: 341594 rows, 170091 columns, 11238525 nonzeros
Variable types: 941 continuous, 169150 integer (91390 binary

In [304]:
# #display solution

# print("Obj: {:.5f}".format(model.objVal))

# model.printAttr('Xn')
model.setParam('SolutionNumber', 0)
model.printAttr('Xn')


    Variable           Xn 
-------------------------
           m      615.016 
    A[1,3,1]            1 
    A[2,2,2]            1 
    A[2,3,3]            1 
    A[1,3,4]            1 
    A[1,1,5]            1 
    X[1,4,5]            1 
    X[1,5,1]            1 
    X[1,0,4]            1 
   X[1,1,-1]            1 
    X[2,3,2]            1 
    X[2,0,3]            1 
   X[2,2,-1]            1 
Yp[1,MyPickNode3_PL1,MyPickNode3_I1]            1 
Yp[1,MyPickNode18_PL4,MyPickNode18_I4]            1 
Yp[1,MyPickNode18_PL5,MyCoopNode10_I5]            1 
Yp[2,MyPickNode5_PL2,MyPickNode1_I2]            1 
Yp[2,MyPickNode4_PL3,MyPickNode2_I3]            1 
Yp[1,MyCoopNode10_I5,MyPickNode3_PL1]            1 
Yp[1,MyPickNode18_I4,MyPickNode18_PL5]            1 
Yp[2,MyPickNode2_I3,MyPickNode5_PL2]            1 
Yp[1,MyHome_P1,MyPickNode18_PL4]            1 
Yp[2,MyHome_P2,MyPickNode4_PL3]            1 
Yp[1,MyPickNode3_I1,MyHome_P1]            1 
Yp[2,MyPickNode1_I2,MyHome_P2]            

tp[1,MyCoopNode9_I5]      397.518 
tp[1,MyCoopNode10_I5]      329.999 
tp[1,MyCoopNode11_I5]      447.517 
tp[1,MyCoopNode12_I5]      437.517 
tp[2,MyPickNode3_PL1]      9870.04 
tp[2,MyPickNode5_PL2]      359.999 
tp[2,MyPickNode4_PL3]           40 
tp[2,MyPickNode18_PL4]      20657.5 
tp[2,MyPickNode18_PL5]         9860 
tp[2,MyPickNode1_I1]      300.016 
tp[2,MyPickNode2_I1]      290.016 
tp[2,MyPickNode3_I1]      270.016 
tp[2,MyPickNode4_I1]      280.016 
tp[2,MyPickNode5_I1]      300.016 
tp[2,MyPickNode6_I1]      310.017 
tp[2,MyPickNode7_I1]      320.016 
tp[2,MyPickNode8_I1]      310.016 
tp[2,MyPickNode9_I1]      310.017 
tp[2,MyPickNode10_I1]      320.017 
tp[2,MyPickNode11_I1]      320.016 
tp[2,MyPickNode12_I1]      330.016 
tp[2,MyPickNode13_I1]      340.016 
tp[2,MyPickNode14_I1]      330.016 
tp[2,MyPickNode15_I1]      330.016 
tp[2,MyPickNode16_I1]      340.015 
tp[2,MyPickNode17_I1]      340.016 
tp[2,MyPickNode18_I1]      350.015 
tp[2,MyPickNode19_I1]      360.011 


tt[1,MyPickNode6_I2]      615.024 
tt[1,MyPickNode7_I2]      640.017 
tt[1,MyPickNode8_I2]      635.017 
tt[1,MyPickNode9_I2]      625.018 
tt[1,MyPickNode10_I2]      620.017 
tt[1,MyPickNode11_I2]      620.021 
tt[1,MyPickNode12_I2]      625.056 
tt[1,MyPickNode13_I2]      650.017 
tt[1,MyPickNode14_I2]      645.017 
tt[1,MyPickNode15_I2]      635.017 
tt[1,MyPickNode16_I2]      630.017 
tt[1,MyPickNode17_I2]      630.021 
tt[1,MyPickNode18_I2]      635.021 
tt[1,MyPickNode19_I2]      660.017 
tt[1,MyPickNode20_I2]      655.017 
tt[1,MyPickNode21_I2]      645.017 
tt[1,MyPickNode22_I2]      640.017 
tt[1,MyPickNode23_I2]      640.021 
tt[1,MyPickNode24_I2]      645.021 
tt[1,MyCoopNode1_I2]      627.519 
tt[1,MyCoopNode2_I2]      612.568 
tt[1,MyCoopNode3_I2]      612.524 
tt[1,MyCoopNode4_I2]      637.517 
tt[1,MyCoopNode5_I2]      622.518 
tt[1,MyCoopNode6_I2]      622.524 
tt[1,MyCoopNode7_I2]      647.517 
tt[1,MyCoopNode8_I2]      632.517 
tt[1,MyCoopNode9_I2]      632.521 
tt[1,

tt[2,MyPickNode12_I4]      9914.96 
tt[2,MyPickNode13_I4]         9885 
tt[2,MyPickNode14_I4]      9895.01 
tt[2,MyPickNode15_I4]         9915 
tt[2,MyPickNode16_I4]         9920 
tt[2,MyPickNode17_I4]         9920 
tt[2,MyPickNode18_I4]         9915 
tt[2,MyPickNode19_I4]         9865 
tt[2,MyPickNode20_I4]         9875 
tt[2,MyPickNode21_I4]         9895 
tt[2,MyPickNode22_I4]         9905 
tt[2,MyPickNode23_I4]      9905.02 
tt[2,MyPickNode24_I4]         9905 
tt[2,MyCoopNode1_I4]         9850 
tt[2,MyCoopNode2_I4]         9880 
tt[2,MyCoopNode3_I4]         9890 
tt[2,MyCoopNode4_I4]         9870 
tt[2,MyCoopNode5_I4]      622.518 
tt[2,MyCoopNode6_I4]      622.524 
tt[2,MyCoopNode7_I4]         9890 
tt[2,MyCoopNode8_I4]       9917.5 
tt[2,MyCoopNode9_I4]       9917.5 
tt[2,MyCoopNode10_I4]         9870 
tt[2,MyCoopNode11_I4]         9900 
tt[2,MyCoopNode12_I4]      9907.49 
tt[2,MyPickNode1_I5]      580.016 
tt[2,MyPickNode2_I5]      585.016 
tt[2,MyPickNode3_I5]      595.016 
tt[2

tt[3,MyPickNode19_I5]      660.017 
tt[3,MyPickNode20_I5]      655.017 
tt[3,MyPickNode21_I5]      645.017 
tt[3,MyPickNode22_I5]      640.017 
tt[3,MyPickNode23_I5]      640.021 
tt[3,MyPickNode24_I5]      645.021 
tt[3,MyCoopNode1_I5]      627.519 
tt[3,MyCoopNode2_I5]      612.568 
tt[3,MyCoopNode3_I5]      612.524 
tt[3,MyCoopNode4_I5]      637.517 
tt[3,MyCoopNode5_I5]      622.518 
tt[3,MyCoopNode6_I5]      622.524 
tt[3,MyCoopNode7_I5]      647.517 
tt[3,MyCoopNode8_I5]      632.517 
tt[3,MyCoopNode9_I5]      632.521 
tt[3,MyCoopNode10_I5]      657.517 
tt[3,MyCoopNode11_I5]      642.517 
tt[3,MyCoopNode12_I5]      642.521 


In [305]:
count = 0
for v in model.getVars():
    if 'X' in v.VarName and v.x==1:
        keys = v.varName[v.varName.index('[')+1:v.varName.index(']')].split(',')
        count= count +1
        print(keys)
print(count)

['1', '4', '5']
['1', '5', '1']
['1', '0', '4']
['1', '1', '-1']
['2', '3', '2']
['2', '0', '3']
['2', '2', '-1']
7


In [332]:
count = 0
for v in model.getVars():
    if 'Yp' in v.VarName and v.x>0.1:
        keys = v.varName[v.varName.index('[')+1:v.varName.index(']')].split(',')
        tp_name = 'tp' + '[' + keys[0]+ ',' + keys[-1] + ']'
        tp_result = model.getVarByName(tp_name)
        count= count +1
        try:
            print(f'{keys,  tp_result.x}')
        except:
            print(f'{keys}')
print(count)

(['1', 'MyPickNode3_PL1', 'MyPickNode3_I1'], 519.9991113720407)
(['1', 'MyPickNode18_PL4', 'MyPickNode18_I4'], 139.99942630494462)
(['1', 'MyPickNode18_PL5', 'MyCoopNode10_I5'], 329.99944996029984)
(['2', 'MyPickNode5_PL2', 'MyPickNode1_I2'], 449.99880361143187)
(['2', 'MyPickNode4_PL3', 'MyPickNode2_I3'], 100.00803769903723)
(['1', 'MyCoopNode10_I5', 'MyPickNode3_PL1'], 489.99911137204066)
(['1', 'MyPickNode18_I4', 'MyPickNode18_PL5'], 214.99942630493928)
(['2', 'MyPickNode2_I3', 'MyPickNode5_PL2'], 359.9992784484584)
(['1', 'MyHome_P1', 'MyPickNode18_PL4'], 109.99942630494462)
(['2', 'MyHome_P2', 'MyPickNode4_PL3'], 40.000003439956345)
['1', 'MyPickNode3_I1', 'MyHome_P1']
['2', 'MyPickNode1_I2', 'MyHome_P2']
12


In [307]:
count = 0
for v in model.getVars():
    if 'A' in v.VarName and v.x==1:
        keys = v.varName[v.varName.index('[')+1:v.varName.index(']')].split(',')
        count= count +1
        print(keys)
print(count)

['1', '3', '1']
['2', '2', '2']
['2', '3', '3']
['1', '3', '4']
['1', '1', '5']
5


In [335]:
count = 0
for v in model.getVars():
    if 'Yt' in v.VarName and v.x>0.1:
        keys = v.varName[v.varName.index('[')+1:v.varName.index(']')].split(',')
        tt_name = 'tt' + '[' + keys[0]+ ',' + keys[-1] + ']'
        tt_result = model.getVarByName(tt_name)
        count= count +1
        try:
            print(f'{keys,  tt_result.x}')
        except:
            print(f'{keys}')
print(count)

(['1', 'MyHome_T1', 'MyCoopNode10_I5'], 359.99944996029984)
(['2', 'MyHome_T2', 'MyPickNode1_I2'], 479.99880361143187)
(['3', 'MyHome_T3', 'MyPickNode18_I4'], 169.99942630494448)
(['1', 'MyCoopNode10_I5', 'MyDepot_1'], 615.0164473499499)
(['2', 'MyPickNode1_I2', 'MyDepot_5'], 615.0164473499499)
(['3', 'MyPickNode3_I1', 'MyDepot_2'], 615.0164474115404)
(['3', 'MyPickNode2_I3', 'MyDepot_9'], 467.49935606173784)
(['3', 'MyDepot_9', 'MyPickNode3_I1'], 549.999111372028)
['1', 'MyDepot_1', 'MyHome_T1']
['3', 'MyDepot_2', 'MyHome_T3']
['2', 'MyDepot_5', 'MyHome_T2']
(['3', 'MyPickNode18_I4', 'MyPickNode2_I3'], 264.99901555390716)
12


In [309]:
count = 0
for v in model.getVars():
    if 'St' in v.VarName and v.x>0:
        keys = v.varName[v.varName.index('[')+1:v.varName.index(']')].split(',')
        count= count +1
        print(f'{keys}, {v.x}')
print(count)

['1', 'MyHome_T1', 'MyCoopNode10_I5'], 1.0
['2', 'MyHome_T2', 'MyPickNode1_I2'], 1.0
['3', 'MyHome_T3', 'MyPickNode18_I4'], 1.0
['3', 'MyDepot_9', 'MyPickNode3_I1'], 1.0
['3', 'MyPickNode18_I4', 'MyPickNode2_I3'], 2.0
5


In [310]:
count = 0
for v in model.getVars():
    if 'tp' in v.VarName and v.x>0:
        keys = v.varName[v.varName.index('[')+1:v.varName.index(']')].split(',')
        count= count +1
        print(f'{keys}, {v.x}')
print(count)

['1', 'MyPickNode3_PL1'], 489.99911137204066
['1', 'MyPickNode5_PL2'], 9850.000040361192
['1', 'MyPickNode4_PL3'], 9870.001021325937
['1', 'MyPickNode18_PL4'], 109.99942630494462
['1', 'MyPickNode18_PL5'], 214.99942630493928
['1', 'MyPickNode1_I1'], 697.5166232299476
['1', 'MyPickNode2_I1'], 687.5169778375639
['1', 'MyPickNode3_I1'], 519.9991113720407
['1', 'MyPickNode4_I1'], 677.5165716409634
['1', 'MyPickNode5_I1'], 697.5169951670056
['1', 'MyPickNode6_I1'], 707.5176503409117
['1', 'MyPickNode7_I1'], 717.5167487204526
['1', 'MyPickNode8_I1'], 707.5165655502302
['1', 'MyPickNode9_I1'], 707.5180017340414
['1', 'MyPickNode10_I1'], 717.5176492541747
['1', 'MyPickNode11_I1'], 717.5167477802557
['1', 'MyPickNode12_I1'], 727.5167665230061
['1', 'MyPickNode13_I1'], 737.5165643021415
['1', 'MyPickNode14_I1'], 727.5167582165668
['1', 'MyPickNode15_I1'], 727.5165531011249
['1', 'MyPickNode16_I1'], 737.5156403743846
['1', 'MyPickNode17_I1'], 737.5167950991527
['1', 'MyPickNode18_I1'], 747.516316

In [315]:
model.getVars().find('A[1,1,1]')

AttributeError: 'list' object has no attribute 'find'

In [24]:
cc = [1,2,3,4,9,0,12]

In [176]:
try:
    cc.remove(1)
except:
    pass

In [2]:
cc = '1_1'
cc.index('_')

1